<a href="https://colab.research.google.com/github/soukhtanlou/iran-map-dashboard/blob/main/IrDevIndextest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
import pandas as pd
import geopandas as gpd
import folium
import ipywidgets as widgets
from IPython.display import display, clear_output

# Load the GeoJSON file
gdf = gpd.read_file('/content/IRN_adm.json')
gdf.crs = 'epsg:4326'

# Load the Excel file
excel_file = pd.ExcelFile('/content/IrDevIndextest.xlsx')

# Load the 'Index' sheet
index_df = excel_file.parse('Index')
sheet_options = index_df.set_index('index code')['index'].to_dict()

# Load the Location ID sheet to get the names
location_df = excel_file.parse('Location ID')
location_dict = location_df.set_index('ID_1')['NAME_1'].to_dict()

# Define dropdowns and checkbox
sheet_selector = widgets.Dropdown(options=list(sheet_options.keys()), value=list(sheet_options.keys())[0], description='Indicator:')
year_selector = widgets.Dropdown(options=['2019', '2020', '2021', '2022', '2023'], value='2023', description='Year:')
reverse_colors_checkbox = widgets.Checkbox(value=False, description='Reverse Colors', disabled=False)

output = widgets.Output()

def update_map(selected_index_code, year, reverse_colors):
    sheet = sheet_options[selected_index_code]
    df = excel_file.parse(sheet)

    temp_df = df[['ID_1', year]]

    # ***THE MISSING PIECE: Merge the data with the GeoDataFrame***
    merged_gdf = gdf.merge(temp_df, on='ID_1', how='left')

    m = folium.Map(location=[32, 53], zoom_start=5)

    fill_color = 'YlGn_r' if reverse_colors else 'YlGn'

    choropleth = folium.Choropleth(
        geo_data=merged_gdf.to_json(),
        name='choropleth',
        data=merged_gdf,
        columns=['ID_1', year],
        key_on='feature.properties.ID_1',
        fill_color=fill_color,
        fill_opacity=0.8,
        line_opacity=0.0,
        legend_name=f'{selected_index_code} - {year}'
    ).add_to(m)

    for feature in choropleth.geojson.data['features']:
        try:
            feature_id = feature['properties']['ID_1']
            feature_name = location_dict.get(feature_id, "Name Not Found")
            data_value = feature['properties'][year] # Access the data value from the merged GeoDataFrame
            popup_text = f"{feature_name}: {data_value:.2f}"

            folium.features.GeoJson(
                feature,
                name=feature_name,
                tooltip=folium.Tooltip(popup_text)
            ).add_to(m)

        except KeyError as e:
            print(f"KeyError: {e}. Feature: {feature}")
            continue

    with output:
        output.clear_output(wait=True)
        display(m)

# Link dropdown menus to update function using interactive and display them
ui = widgets.HBox([sheet_selector, year_selector, reverse_colors_checkbox])
out = widgets.interactive_output(
    update_map,
    {
        'selected_index_code': sheet_selector,
        'year': year_selector,
        'reverse_colors': reverse_colors_checkbox
    }
)
display(ui, out)

# Call the function initially to display the map with default values
update_map(sheet_selector.value, year_selector.value, reverse_colors_checkbox.value)
display(output)

Output()

Output()